In [1]:
from sklearn.datasets import load_iris # scikit-learn의 샘플 데이터 로드를 위해 import
import numpy as np

In [2]:
iris = load_iris()
x, y = iris.data, iris.target

In [3]:
print(iris.data)
print(iris.feature_names)

# label
print(iris.target)
print(iris.target_names)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.

In [4]:
def get_gini(label):
    sample_length = len(label)
    target_len = len(np.unique(label))
    count_arr = np.zeros((target_len))

    for i in range(target_len):
        lenth = len(label[label == np.unique(label)[i]])
        count_arr[i] = lenth
    result = (1 - np.sum((count_arr/sample_length)**2))
    return result

def get_gini_multinode(series):
    len_leaf = len(series)

    list_gini = np.zeros((len_leaf))
    sample_num = 0
    for i in range(len_leaf):
        subset_len = len(series[i])
        temp = get_gini(series[i])*subset_len
        list_gini[i] = temp
        sample_num = sample_num + subset_len
    result = np.sum(list_gini/sample_num)
    return result
    
    

In [118]:
get_gini(iris.target[:150])

0.6666666666666667

In [119]:
split_a = y[x[:,0] >= 3]
split_b = y[x[:,0] < 3]
get_gini_multinode([split_a,split_b])

0.6666666666666667

In [142]:
def search_th(sample,label,count = 0):

    count += 1
    tree_node = {
        "k":None,
        "th":None,
        "node_left":None,
        "node_right":None,
    }

    k_len = len(np.unique(label))
    trials = 30
    results = []

    for i in range(k_len):
        max = np.max(sample[:,i])
        min = np.min(sample[:,i])

        for j in range(trials):
            th = np.random.uniform(min,max,1)[0]
            split_a_label = label[sample[:,i] >= th]
            split_b_label = label[sample[:,i] < th]
            gini_pre = get_gini(label)
            gini_post = get_gini_multinode([split_a_label,split_b_label])

            if gini_post < gini_pre:
                results.append([i,th,gini_post])

    try:        
        results = np.array(results)
        out = results[results[:,2].argmin()]
    except:
        tree_node = {
            # "sample":[sample,label]
            "label":np.unique(label)[0]
        }
        return tree_node
    
    left_label = label[sample[:,int(out[0])] >= out[1]]
    left_input = sample[sample[:,int(out[0])] >= out[1]]
    right_label = label[sample[:,int(out[0])] < out[1]]
    right_input = sample[sample[:,int(out[0])] < out[1]]
    # right_label = label[sample[:,int(out[0])] >= out[1]]
    # right_input = sample[sample[:,int(out[0])] >= out[1]]

    

    # if count > 3:
    #     tree_node["k"] = out[0]
    #     tree_node["th"] = out[1]
    #     tree_node["node_left"] = [left_input,left_label]
    #     tree_node["node_right"] = [right_input,right_label]
        
    #     return tree_node
    

    tree_node["k"] = out[0]
    tree_node["th"] = out[1]
    tree_node["node_left"] = search_th(left_input,left_label,count)
    tree_node["node_right"] = search_th(right_input,right_label,count)

    return tree_node
        
        


        

In [143]:
test = search_th(x,y)

In [144]:
test['node_left']['node_left']

{'k': 0.0,
 'th': 7.073081091309683,
 'node_left': {'label': 2},
 'node_right': {'k': 1.0,
  'th': 2.394543268287786,
  'node_left': {'k': 0.0,
   'th': 6.138824213918811,
   'node_left': {'k': 0.0,
    'th': 6.9776077134398875,
    'node_left': {'label': 1},
    'node_right': {'k': 1.0,
     'th': 3.1086732162842883,
     'node_left': {'k': 0.0,
      'th': 6.4324576287232444,
      'node_left': {'label': 2},
      'node_right': {'k': 1.0,
       'th': 3.380958566950581,
       'node_left': {'label': 2},
       'node_right': {'label': 1}}},
     'node_right': {'k': 0.0,
      'th': 6.537414250481418,
      'node_left': {'k': 0.0,
       'th': 6.693503529898918,
       'node_left': {'k': 1.0,
        'th': 2.678244074675519,
        'node_left': {'k': 1.0,
         'th': 2.9085294168243454,
         'node_left': {'k': 0.0,
          'th': 6.793809465122317,
          'node_left': {'k': 0.0,
           'th': 6.897809850628281,
           'node_left': {'label': 1},
           'node_right